In [3]:
# import numpy as np
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import plotly.offline as pyo
# from sklearn.linear_model import LinearRegression
# import polars as pl
# import gc

# # Load and prepare data
# transactions = pd.read_parquet("../data/transactions_cleaned.parquet")


In [4]:

# # Standardize column names for consistency
# column_mapping = {
#     'Store Country': 'Store_Country',
#     'Store City': 'Store_City', 
#     'Employee ID': 'Employee_ID',
#     'Customer ID': 'Customer_ID',
#     'Product ID': 'Product_ID',
#     'Invoice USD': 'Invoice_USD',
#     'Unit Price': 'Unit_Price'
# }

# # Apply column mapping if columns exist
# for old_name, new_name in column_mapping.items():
#     if old_name in transactions.columns:
#         transactions.rename(columns={old_name: new_name}, inplace=True)

# # Data preparation and cleaning
# def prepare_data():
#     """Prepare and clean all datasets for analysis"""
    
#     # Geographic Sales Data
#     geo_sales = (
#         transactions
#         .dropna(subset=['Store_Country', 'Store_City', 'Latitude', 'Longitude', 'Invoice_USD'])
#         .groupby(['Store_Country', 'Store_City', 'Latitude', 'Longitude'], as_index=False)
#         .agg({'Invoice_USD': 'sum'})
#         .rename(columns={'Invoice_USD': 'Total_Sales_USD'})
#     )
#     geo_sales.to_csv("data/geo_sales_summary.csv", index=False)
    
#     # Employee Performance Data
#     employee_perf = transactions.groupby("Employee_ID", as_index=False).agg(
#         Total_Sales=("Invoice_USD", "sum"),
#         Total_Quantity=("Quantity", "sum"),
#         Transaction_Count=("Invoice_USD", "count")
#     )
#     employee_perf["Avg_Price"] = employee_perf["Total_Sales"] / employee_perf["Total_Quantity"]
#     employee_perf = employee_perf[employee_perf["Total_Quantity"] >= 5]  # Filter minimum activity
#     employee_perf.to_csv("data/employee_performance.csv", index=False)
    
#     # Country Performance Data
#     country_perf = transactions.groupby("Store_Country", as_index=False).agg(
#         Total_Sales=("Invoice_USD", "sum"),
#         Total_Quantity=("Quantity", "sum"),
#         Transaction_Count=("Invoice_USD", "count")
#     )
#     country_perf["Avg_Price"] = country_perf["Total_Sales"] / country_perf["Total_Quantity"]
#     country_perf.to_csv("data/country_performance.csv", index=False)
    
#     # City Performance Data
#     city_perf = transactions.groupby("Store_City", as_index=False).agg(
#         Total_Sales=("Invoice_USD", "sum"),
#         Total_Quantity=("Quantity", "sum"),
#         Transaction_Count=("Invoice_USD", "count")
#     )
#     city_perf["Avg_Price"] = city_perf["Total_Sales"] / city_perf["Total_Quantity"]
#     city_perf.to_csv("data/city_performance.csv", index=False)
    
#     # Customer Performance Data
#     customer_perf = transactions.groupby("Customer_ID", as_index=False).agg(
#         Total_Sales=("Invoice_USD", "sum"),
#         Total_Quantity=("Quantity", "sum"),
#         Transaction_Count=("Invoice_USD", "count")
#     )
#     customer_perf["Avg_Price"] = customer_perf["Total_Sales"] / customer_perf["Total_Quantity"]
#     customer_perf = customer_perf[
#         (customer_perf["Total_Quantity"] > 0) &
#         (customer_perf["Total_Sales"] > 0) &
#         customer_perf["Avg_Price"].notna()
#     ]
#     customer_perf.to_csv("data/customer_performance.csv", index=False)
    
#     # Product Performance Data
#     product_perf = transactions.groupby("Product_ID", as_index=False).agg(
#         Total_Sales=("Invoice_USD", "sum"),
#         Total_Quantity=("Quantity", "sum"),
#         Transaction_Count=("Invoice_USD", "count")
#     )
#     product_perf["Avg_Price"] = product_perf["Total_Sales"] / product_perf["Total_Quantity"]
#     product_perf = product_perf[
#         (product_perf["Total_Quantity"] > 0) &
#         (product_perf["Total_Sales"] > 0) &
#         product_perf["Avg_Price"].notna()
#     ]
#     product_perf.to_csv("data/product_performance.csv", index=False)
    
#     # Discount Analysis Data
#     transactions['Net_Revenue'] = transactions['Quantity'] * (
#         transactions['Unit_Price'] * (1 - transactions['Discount'])
#     )
    
#     discount_analysis = transactions.groupby('Product_ID').agg(
#         Avg_Discount=('Discount', 'mean'),
#         Total_Quantity=('Quantity', 'sum'),
#         Net_Revenue=('Net_Revenue', 'sum'),
#         Total_Sales=('Invoice_USD', 'sum')
#     ).reset_index()
#     discount_analysis.to_csv("data/discount_analysis.csv", index=False)
    
#     # Customer Segmentation (RFM Analysis)
#     # Assuming we have date column for recency calculation
#     if 'Date' in transactions.columns:
#         current_date = transactions['Date'].max()
#         rfm_data = transactions.groupby('Customer_ID').agg(
#             Recency=('Date', lambda x: (current_date - x.max()).days),
#             Frequency=('Invoice_USD', 'count'),
#             Monetary=('Invoice_USD', 'sum')
#         ).reset_index()
        
#         # RFM Scoring
#         rfm_data['R_Score'] = pd.qcut(rfm_data['Recency'], 5, labels=[5,4,3,2,1])
#         rfm_data['F_Score'] = pd.qcut(rfm_data['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
#         rfm_data['M_Score'] = pd.qcut(rfm_data['Monetary'], 5, labels=[1,2,3,4,5])
#         rfm_data['RFM_Score'] = rfm_data['R_Score'].astype(str) + rfm_data['F_Score'].astype(str) + rfm_data['M_Score'].astype(str)
        
#         # Customer Segments
#         def segment_customers(row):
#             if row['RFM_Score'] in ['555', '554', '544', '545', '454', '455', '445']:
#                 return 'Champions'
#             elif row['RFM_Score'] in ['543', '444', '435', '355', '354', '345', '344', '335']:
#                 return 'Loyal Customers'
#             elif row['RFM_Score'] in ['512', '511', '422', '421', '412', '411', '311']:
#                 return 'Potential Loyalists'
#             elif row['RFM_Score'] in ['533', '532', '531', '523', '522', '521', '515', '514', '513', '425', '424', '413', '414', '415', '315', '314', '313']:
#                 return 'New Customers'
#             elif row['RFM_Score'] in ['155', '154', '144', '214', '215', '115', '114']:
#                 return 'At Risk'
#             elif row['RFM_Score'] in ['255', '254', '245', '244', '253', '252', '243', '242', '235', '234', '225', '224', '153', '152', '145', '143', '142', '135', '134', '125', '124']:
#                 return 'Cannot Lose Them'
#             else:
#                 return 'Others'
        
#         rfm_data['Customer_Segment'] = rfm_data.apply(segment_customers, axis=1)
#         rfm_data.to_csv("data/rfm_analysis.csv", index=False)
    
#     print("✅ All data preparation completed successfully!")

# # Execute data preparation
# prepare_data()


In [5]:
# import numpy as np
# import pandas as pd
# import polars as pl
# from datetime import datetime, timedelta
# import gc
# import os

# # Create data directory if it doesn't exist
# os.makedirs("data", exist_ok=True)

# # Load data with Polars for better memory efficiency
# print("📊 Loading data with Polars for memory efficiency...")
# # transactions_pl = pl.read_parquet("data/transactions_cleaned.parquet")
# transactions_pl = pl.from_pandas(transactions)



In [14]:
import polars as pl
import pandas as pd
from datetime import datetime
import os

# Exchange rates you provided
exchange_rates = {
    'EUR': 1.13,
    'GBP': 1.34,
    'CNY': 0.14,
    'USD': 1.0
}

city_translation_map = {

}

country_translation_map = {
    '中国': 'China',
    'España': 'Spain',
    'Portugal': 'Portugal',
    'Deutschland': 'Germany',
    'France': 'France',
    'United Kingdom': 'United Kingdom',
    'United States': 'United States'
}

def load_data_efficiently():
    """Load data with optimized memory usage"""
    print("📊 Loading data efficiently...")
    
    # Load with pandas first, then convert to polars for better performance
    print("Loading discounts...")
    discounts = pd.read_csv(r'C:\Users\Alan\Downloads\BDM\BDM_raw_data\discounts.csv')
    
    print("Loading products...")
    products = pd.read_csv(r'C:\Users\Alan\Downloads\BDM\BDM_raw_data\products.csv')
    
    print("Loading employees...")
    employees = pd.read_csv(r'C:\Users\Alan\Downloads\BDM\BDM_raw_data\employees.csv')
    
    print("Loading stores...")
    stores = pd.read_csv(r'C:\Users\Alan\Downloads\BDM\BDM_raw_data\stores.csv')
    
    print("Loading customers...")
    customers = pd.read_csv(r'C:\Users\Alan\Downloads\BDM\BDM_raw_data\customers.csv', low_memory=False)
    
    print("Loading transactions (this may take a moment)...")
    transactions = pd.read_csv(r'C:\Users\Alan\Downloads\BDM\BDM_raw_data\transactions.csv', low_memory=False)
    print(f"Intial Transactions loaded: {len(transactions)} rows")
    transactions=transactions.drop_duplicates()
    print(f"Final Transactions loaded: {len(transactions)} rows")
    
    # Convert to polars for better performance
    print("Converting to Polars for better performance...")
    discounts_pl = pl.from_pandas(discounts)
    products_pl = pl.from_pandas(products)
    employees_pl = pl.from_pandas(employees)
    stores_pl = pl.from_pandas(stores)
    customers_pl = pl.from_pandas(customers)
    transactions_pl = pl.from_pandas(transactions)
    
    print(f"✅ Data loaded successfully!")
    print(f"Transactions shape: {transactions_pl.shape}")
    
    return discounts_pl, products_pl, employees_pl, stores_pl, customers_pl, transactions_pl


def clean_and_translate_data(master_data, city_translation_map, country_translation_map):
    """
    Clean and translate city and country names into English using Polars DataFrame.
    Compatible with older Polars versions.
    """
    master_data = master_data.with_columns([
        pl.col("City").replace(city_translation_map).alias("City"),
        pl.col("Country").replace(country_translation_map).alias("Country")
    ])
    return master_data


def clean_date_columns(df, date_columns):
    """Clean and convert date columns to simple YYYY-MM-DD format"""
    print(f"🗓️ Converting date columns to YYYY-MM-DD format: {date_columns}")
    
    for col in date_columns:
        if col in df.columns:
            try:
                # Convert to date (not datetime) - this removes time component automatically
                df = df.with_columns(
                    pl.col(col).str.strptime(pl.Date, format="%Y-%m-%d", strict=False)
                    .fill_null(
                        pl.col(col).str.strptime(pl.Date, format="%Y-%m-%d %H:%M:%S", strict=False)
                    )
                    .fill_null(
                        pl.col(col).str.strptime(pl.Date, format="%d/%m/%Y", strict=False)
                    )
                    .alias(col)
                )
                print(f"✅ Converted {col} to YYYY-MM-DD date format")
            except Exception as e:
                print(f"❌ Could not convert {col}: {e}")
                # Keep original if conversion fails
                pass
    
    return df

def prepare_lookup_tables(discounts_pl, products_pl, employees_pl, stores_pl, customers_pl):
    """Prepare minimal lookup tables with only essential information"""
    print("🔧 Preparing optimized lookup tables...")
    
    # Clean discount dates - Start and End to YYYY-MM-DD
    discounts_clean = clean_date_columns(discounts_pl, ['Start', 'End'])
    discounts_lookup = discounts_clean.select([
        'Start', 'End', 'Discont', 'Description', 'Category', 'Sub Category'
    ])
    
    # Products - keep essential info only
    products_lookup = products_pl.select([
        'Product ID', 'Category', 'Sub Category', 'Description EN', 
        'Color', 'Sizes', 'Production Cost'
    ])
    
    # Employees - minimal info
    employees_lookup = employees_pl.select([
        'Employee ID', 'Store ID', 'Name', 'Position'
    ])
    
    # Stores - essential location info
    stores_pl = clean_and_translate_data(stores_pl, city_translation_map, country_translation_map)
    stores_lookup = stores_pl.select([
        'Store ID', 'Country', 'City', 'Store Name', 'Number of Employees'
    ])
    
    # Customers - clean birth date to YYYY-MM-DD and keep essential info
    customers_pl = clean_and_translate_data(customers_pl, city_translation_map, country_translation_map)
    customers_clean = clean_date_columns(customers_pl, ['Date Of Birth'])
    customers_lookup = customers_clean.select([
        'Customer ID', 'Name', 'Email', 'Gender', 'Date Of Birth', 'Job Title'
    ])
    
    print("✅ Lookup tables prepared")
    return discounts_lookup, products_lookup, employees_lookup, stores_lookup, customers_lookup

def process_transactions(transactions_pl, exchange_rates):
    """Process transactions with USD conversion and date cleaning"""
    print("💰 Processing transactions...")
    
    # Clean transaction dates - convert Date to YYYY-MM-DD format (removes time)
    
    transactions_clean = clean_date_columns(transactions_pl, ['Date'])
    
    # Add exchange rate column
    print("💱 Adding USD conversion...")
    transactions_clean = transactions_clean.with_columns(
        pl.col('Currency').map_elements(
            lambda x: exchange_rates.get(x, 1.0), 
            return_dtype=pl.Float64
        ).alias("Exchange_Rate_to_USD")
    )
    
    # Convert monetary columns to USD
    monetary_columns = ['Unit Price', 'Line Total', 'Invoice Total']
    
    for col in monetary_columns:
        if col in transactions_clean.columns:
            usd_col_name = f"{col.replace(' ', '_')}_USD"
            transactions_clean = transactions_clean.with_columns(
                (pl.col(col) * pl.col("Exchange_Rate_to_USD")).alias(usd_col_name)
            )
            print(f"✅ Created {usd_col_name}")
    
    return transactions_clean

def create_master_dataset():
    """Create the master dataset with optimized merging"""
    print("🏗️ Creating master dataset...")
    
    # Load data
    discounts_pl, products_pl, employees_pl, stores_pl, customers_pl, transactions_pl = load_data_efficiently()
    
    # Prepare lookup tables
    discounts_lookup, products_lookup, employees_lookup, stores_lookup, customers_lookup = prepare_lookup_tables(
        discounts_pl, products_pl, employees_pl, stores_pl, customers_pl
    )
    
    # Process transactions
    transactions_processed = process_transactions(transactions_pl, exchange_rates)
    
    # Merge with lookup tables (left joins to keep all transactions)
    print("🔗 Merging tables...")
    
    # Merge with products
    master_data = transactions_processed.join(
        products_lookup, 
        on='Product ID', 
        how='left'
    )
    print(f"✅ Merged with products. Shape: {master_data.shape}")
    
    # Merge with customers
    master_data = master_data.join(
        customers_lookup, 
        on='Customer ID', 
        how='left'
    )
    print(f"✅ Merged with customers. Shape: {master_data.shape}")
    
    # Merge with stores
    master_data = master_data.join(
        stores_lookup, 
        on='Store ID', 
        how='left'
    )
    print(f"✅ Merged with stores. Shape: {master_data.shape}")
    
    # Merge with employees
    master_data = master_data.join(
        employees_lookup, 
        on='Employee ID', 
        how='left'
    )
    print(f"✅ Merged with employees. Shape: {master_data.shape}")
    
    # Create data directory if it doesn't exist
    os.makedirs("data", exist_ok=True)
    
    # Save lookup tables separately for future use
    print("💾 Saving lookup tables...")
    discounts_lookup.write_parquet("data/discounts_lookup.parquet")
    products_lookup.write_parquet("data/products_lookup.parquet")
    employees_lookup.write_parquet("data/employees_lookup.parquet")
    stores_lookup.write_parquet("data/stores_lookup.parquet")
    customers_lookup.write_parquet("data/customers_lookup.parquet")
    
    # Save master dataset
    print("💾 Saving master dataset...")
    master_data.write_parquet("data/master_transactions.parquet")
    
    # Save a sample CSV for inspection (first 10k rows)
    master_data.head(10000).write_csv("data/master_transactions_sample.csv")
    
    print("✅ Master dataset created successfully!")
    print(f"Final shape: {master_data.shape}")
    print(f"Memory usage optimized by keeping only essential columns from lookup tables")
    
    return master_data, discounts_lookup

def show_dataset_summary(master_data):
    """Show summary of the final dataset"""
    print("\n📊 MASTER DATASET SUMMARY:")
    print(f"Shape: {master_data.shape}")
    print(f"Columns: {len(master_data.columns)}")
    
    print("\n📋 Column List:")
    for i, col in enumerate(master_data.columns, 1):
        print(f"{i:2d}. {col}")
    
    print(f"\n💰 USD Converted Columns:")
    usd_cols = [col for col in master_data.columns if 'USD' in col]
    for col in usd_cols:
        print(f"   • {col}")
    
    print(f"\n🗓️ Date Columns (YYYY-MM-DD format):")
    date_cols = [col for col in master_data.columns if master_data[col].dtype == pl.Date]
    for col in date_cols:
        print(f"   • {col}")
        # Show sample dates
        sample_dates = master_data.select(col).drop_nulls().head(3)
        print(f"     Sample: {sample_dates.to_series().to_list()}")
    
    print(f"\n📈 Sample Data:")
    print(master_data.head(3))
    
    return master_data

# Execute the pipeline
print("🚀 Starting Optimized Data Pipeline...")
master_data, discounts_lookup = create_master_dataset()
final_data = show_dataset_summary(master_data)

print("\n✅ Pipeline Complete!")
print("Files saved:")
print("   • data/master_transactions.parquet (main dataset)")
print("   • data/master_transactions_sample.csv (sample for inspection)")
print("   • data/discounts_lookup.parquet (for discount analysis)")
print("   • data/products_lookup.parquet")
print("   • data/employees_lookup.parquet") 
print("   • data/stores_lookup.parquet")
print("   • data/customers_lookup.parquet")
print("\n📅 All dates are now in simple YYYY-MM-DD format!")


🚀 Starting Optimized Data Pipeline...
🏗️ Creating master dataset...
📊 Loading data efficiently...
Loading discounts...
Loading products...
Loading employees...
Loading stores...
Loading customers...
Loading transactions (this may take a moment)...
Intial Transactions loaded: 6416827 rows
Final Transactions loaded: 6416029 rows
Converting to Polars for better performance...
✅ Data loaded successfully!
Transactions shape: (6416029, 19)
🔧 Preparing optimized lookup tables...
🗓️ Converting date columns to YYYY-MM-DD format: ['Start', 'End']
✅ Converted Start to YYYY-MM-DD date format
✅ Converted End to YYYY-MM-DD date format
🗓️ Converting date columns to YYYY-MM-DD format: ['Date Of Birth']
✅ Converted Date Of Birth to YYYY-MM-DD date format
✅ Lookup tables prepared
💰 Processing transactions...
🗓️ Converting date columns to YYYY-MM-DD format: ['Date']
✅ Converted Date to YYYY-MM-DD date format
💱 Adding USD conversion...
✅ Created Unit_Price_USD
✅ Created Line_Total_USD
✅ Created Invoice_To

In [41]:
# Add this line before loading your data
import gc; gc.collect(); print(f"🧹 Memory cleared: {gc.collect()} objects")


🧹 Memory cleared: 0 objects


In [39]:
sale_total = final_data.filter(pl.col("Transaction Type") == "Sale")["Line_Total_USD"].sum()

# Filter rows for 'Return' and calculate the sum of 'Line_Total_USD'
return_total = final_data.filter(pl.col("Transaction Type") == "Return")["Line_Total_USD"].sum()

# Print the results
print(f"Total USD for Sale: {sale_total}")
print(f"Total USD for Return: {return_total}")

# # Print the results
# print(sum_by_category)


Total USD for Sale: 305884836.54719996
Total USD for Return: -17136183.740900002


In [11]:
import polars as pl

# Assuming final_data is your Polars DataFrame
# Step 1: Filter rows where the transaction type is 'Sale'
sale_data = final_data.filter(pl.col("Transaction Type") == "Sale")

# Step 2: Remove duplicate InvoiceID values
unique_sale_data = sale_data.unique(subset=["Invoice ID"])

# Step 3: Sum the Line_Total_USD column
line_total_sum = unique_sale_data["Invoice_Total_USD"].sum()

# Print the result
print(f"Sum of Line_Total_USD for unique InvoiceIDs (Sale transactions): {line_total_sum}")


Sum of Line_Total_USD for unique InvoiceIDs (Sale transactions): 305884634.1774


In [12]:
import polars as pl

# Assuming final_data is your Polars DataFrame
# Step 1: Filter rows where the transaction type is 'Sale'
sale_data2 = final_data.filter(pl.col("Transaction Type") == "Return")

# Step 2: Remove duplicate InvoiceID values
unique_sale_data2 = sale_data2.unique(subset=["Invoice ID"])

# Step 3: Sum the Line_Total_USD column
line_total_sum2 = unique_sale_data2["Invoice_Total_USD"].sum()

# Print the result
print(f"Sum of Line_Total_USD for unique InvoiceIDs (Return transactions): {line_total_sum2}")


Sum of Line_Total_USD for unique InvoiceIDs (Return transactions): -15683410.0896


In [7]:
def perform_descriptive_statistics(discounts_pl, products_pl, employees_pl, stores_pl, customers_pl, transactions_pl):
    """
    Perform comprehensive descriptive statistics on all datasets
    """
    print("\n" + "="*60)
    print("📊 PERFORMING DESCRIPTIVE STATISTICS ANALYSIS")
    print("="*60)
    
    import os
    from datetime import datetime
    
    # Create output directory
    stats_dir = "descriptive_statistics"
    os.makedirs(stats_dir, exist_ok=True)
    
    stats_summary = {}
    
    # 1. DISCOUNTS DATASET ANALYSIS
    print("\n🏷️ DISCOUNTS Dataset Analysis:")
    print("-" * 40)
    
    discounts_stats = {
        'total_records': discounts_pl.shape[0],
        'total_columns': discounts_pl.shape[1],
        'columns': discounts_pl.columns,
        'data_types': dict(zip(discounts_pl.columns, [str(dtype) for dtype in discounts_pl.dtypes])),
        'null_counts': discounts_pl.null_count().to_pandas().iloc[0].to_dict(),
        'memory_usage_mb': discounts_pl.estimated_size('mb')
    }
    
    # Numeric columns analysis for discounts
    numeric_cols = discounts_pl.select(pl.col(pl.NUMERIC_DTYPES)).columns
    if numeric_cols:
        discounts_stats['numeric_summary'] = discounts_pl.select(numeric_cols).describe().to_pandas().to_dict()
    
    stats_summary['discounts'] = discounts_stats
    
    print(f"   📋 Records: {discounts_stats['total_records']:,}")
    print(f"   📊 Columns: {discounts_stats['total_columns']}")
    print(f"   💾 Memory: {discounts_stats['memory_usage_mb']:.2f} MB")
    
    # 2. PRODUCTS DATASET ANALYSIS
    print("\n📦 PRODUCTS Dataset Analysis:")
    print("-" * 40)
    
    products_stats = {
        'total_records': products_pl.shape[0],
        'total_columns': products_pl.shape[1],
        'columns': products_pl.columns,
        'data_types': dict(zip(products_pl.columns, [str(dtype) for dtype in products_pl.dtypes])),
        'null_counts': products_pl.null_count().to_pandas().iloc[0].to_dict(),
        'memory_usage_mb': products_pl.estimated_size('mb')
    }
    
    # Unique categories/brands analysis
    if 'Category' in products_pl.columns:
        products_stats['unique_categories'] = products_pl['Category'].n_unique()
        products_stats['category_distribution'] = products_pl.group_by('Category').len().sort('len', descending=True).to_pandas().to_dict('records')
    
    # Price analysis if available
    price_cols = [col for col in products_pl.columns if 'price' in col.lower() or 'cost' in col.lower()]
    if price_cols:
        products_stats['price_summary'] = products_pl.select(price_cols).describe().to_pandas().to_dict()
    
    stats_summary['products'] = products_stats
    
    print(f"   📋 Records: {products_stats['total_records']:,}")
    print(f"   📊 Columns: {products_stats['total_columns']}")
    print(f"   💾 Memory: {products_stats['memory_usage_mb']:.2f} MB")
    if 'unique_categories' in products_stats:
        print(f"   🏷️ Categories: {products_stats['unique_categories']}")
    
    # 3. EMPLOYEES DATASET ANALYSIS
    print("\n👥 EMPLOYEES Dataset Analysis:")
    print("-" * 40)
    
    employees_stats = {
        'total_records': employees_pl.shape[0],
        'total_columns': employees_pl.shape[1],
        'columns': employees_pl.columns,
        'data_types': dict(zip(employees_pl.columns, [str(dtype) for dtype in employees_pl.dtypes])),
        'null_counts': employees_pl.null_count().to_pandas().iloc[0].to_dict(),
        'memory_usage_mb': employees_pl.estimated_size('mb')
    }
    
    # Department/Role analysis if available
    dept_cols = [col for col in employees_pl.columns if any(keyword in col.lower() for keyword in ['department', 'role', 'position', 'title'])]
    for col in dept_cols:
        employees_stats[f'{col}_distribution'] = employees_pl.group_by(col).len().sort('len', descending=True).to_pandas().to_dict('records')
    
    stats_summary['employees'] = employees_stats
    
    print(f"   📋 Records: {employees_stats['total_records']:,}")
    print(f"   📊 Columns: {employees_stats['total_columns']}")
    print(f"   💾 Memory: {employees_stats['memory_usage_mb']:.2f} MB")
    
    # 4. STORES DATASET ANALYSIS
    print("\n🏪 STORES Dataset Analysis:")
    print("-" * 40)
    
    stores_stats = {
        'total_records': stores_pl.shape[0],
        'total_columns': stores_pl.shape[1],
        'columns': stores_pl.columns,
        'data_types': dict(zip(stores_pl.columns, [str(dtype) for dtype in stores_pl.dtypes])),
        'null_counts': stores_pl.null_count().to_pandas().iloc[0].to_dict(),
        'memory_usage_mb': stores_pl.estimated_size('mb')
    }
    
    # Location analysis if available
    location_cols = [col for col in stores_pl.columns if any(keyword in col.lower() for keyword in ['city', 'state', 'country', 'region'])]
    for col in location_cols:
        if col in stores_pl.columns:
            stores_stats[f'{col}_distribution'] = stores_pl.group_by(col).len().sort('len', descending=True).to_pandas().to_dict('records')
    
    stats_summary['stores'] = stores_stats
    
    print(f"   📋 Records: {stores_stats['total_records']:,}")
    print(f"   📊 Columns: {stores_stats['total_columns']}")
    print(f"   💾 Memory: {stores_stats['memory_usage_mb']:.2f} MB")
    
    # 5. CUSTOMERS DATASET ANALYSIS
    print("\n👤 CUSTOMERS Dataset Analysis:")
    print("-" * 40)
    
    customers_stats = {
        'total_records': customers_pl.shape[0],
        'total_columns': customers_pl.shape[1],
        'columns': customers_pl.columns,
        'data_types': dict(zip(customers_pl.columns, [str(dtype) for dtype in customers_pl.dtypes])),
        'null_counts': customers_pl.null_count().to_pandas().iloc[0].to_dict(),
        'memory_usage_mb': customers_pl.estimated_size('mb')
    }
    
    # Demographics analysis
    demo_cols = [col for col in customers_pl.columns if any(keyword in col.lower() for keyword in ['gender', 'age', 'city', 'state', 'country'])]
    for col in demo_cols:
        if col in customers_pl.columns:
            customers_stats[f'{col}_distribution'] = customers_pl.group_by(col).len().sort('len', descending=True).head(10).to_pandas().to_dict('records')
    
    stats_summary['customers'] = customers_stats
    
    print(f"   📋 Records: {customers_stats['total_records']:,}")
    print(f"   📊 Columns: {customers_stats['total_columns']}")
    print(f"   💾 Memory: {customers_stats['memory_usage_mb']:.2f} MB")
    
    # 6. TRANSACTIONS DATASET ANALYSIS (Most Important)
    print("\n💳 TRANSACTIONS Dataset Analysis:")
    print("-" * 40)
    
    transactions_stats = {
        'total_records': transactions_pl.shape[0],
        'total_columns': transactions_pl.shape[1],
        'columns': transactions_pl.columns,
        'data_types': dict(zip(transactions_pl.columns, [str(dtype) for dtype in transactions_pl.dtypes])),
        'null_counts': transactions_pl.null_count().to_pandas().iloc[0].to_dict(),
        'memory_usage_mb': transactions_pl.estimated_size('mb')
    }
    
    # Financial metrics
    financial_cols = [col for col in transactions_pl.columns if any(keyword in col.lower() for keyword in ['amount', 'total', 'price', 'cost', 'revenue', 'quantity'])]
    if financial_cols:
        transactions_stats['financial_summary'] = transactions_pl.select(financial_cols).describe().to_pandas().to_dict()
    
    # Date range analysis
    date_cols = [col for col in transactions_pl.columns if any(keyword in col.lower() for keyword in ['date', 'time'])]
    for col in date_cols:
        if col in transactions_pl.columns:
            try:
                date_stats = transactions_pl.select([
                    pl.col(col).min().alias('min_date'),
                    pl.col(col).max().alias('max_date'),
                    pl.col(col).n_unique().alias('unique_dates')
                ]).to_pandas().iloc[0].to_dict()
                transactions_stats[f'{col}_range'] = date_stats
            except:
                pass
    
    # Unique counts for key columns
    key_cols = ['Customer ID', 'Product ID', 'Store ID', 'Employee ID']
    for col in key_cols:
        if col in transactions_pl.columns:
            transactions_stats[f'unique_{col.lower().replace(" ", "_")}'] = transactions_pl[col].n_unique()
    
    stats_summary['transactions'] = transactions_stats
    
    print(f"   📋 Records: {transactions_stats['total_records']:,}")
    print(f"   📊 Columns: {transactions_stats['total_columns']}")
    print(f"   💾 Memory: {transactions_stats['memory_usage_mb']:.2f} MB")
    
    # 7. OVERALL SUMMARY
    print("\n📈 OVERALL DATA SUMMARY:")
    print("-" * 40)
    
    total_records = sum([stats['total_records'] for stats in stats_summary.values()])
    total_memory = sum([stats['memory_usage_mb'] for stats in stats_summary.values()])
    
    overall_stats = {
        'total_datasets': len(stats_summary),
        'total_records_across_all': total_records,
        'total_memory_usage_mb': total_memory,
        'analysis_timestamp': datetime.now().isoformat()
    }
    
    stats_summary['overall'] = overall_stats
    
    print(f"   📊 Total Datasets: {overall_stats['total_datasets']}")
    print(f"   📋 Total Records: {overall_stats['total_records_across_all']:,}")
    print(f"   💾 Total Memory: {overall_stats['total_memory_usage_mb']:.2f} MB")
    
    # 8. SAVE RESULTS
    print(f"\n💾 Saving descriptive statistics...")
    
    # Save as JSON
    import json
    with open(f"{stats_dir}/descriptive_statistics_summary.json", 'w') as f:
        json.dump(stats_summary, f, indent=2, default=str)
    
    # Save detailed CSV reports for each dataset
    for dataset_name, stats in stats_summary.items():
        if dataset_name != 'overall':
            stats_df = pd.DataFrame([stats])
            stats_df.to_csv(f"{stats_dir}/{dataset_name}_statistics.csv", index=False)
    
    print(f"✅ Descriptive statistics completed!")
    print(f"📁 Results saved in: {stats_dir}/")
    print("="*60)
    
    return stats_summary

# Usage in your main function:
def load_data_and_analyze():
    """Load data and perform descriptive statistics"""
    
    # Load data (your existing function)
    discounts_pl, products_pl, employees_pl, stores_pl, customers_pl, transactions_pl = load_data_efficiently()
    
    # Perform descriptive statistics
    stats_summary = perform_descriptive_statistics(
        discounts_pl, products_pl, employees_pl, stores_pl, customers_pl, transactions_pl
    )
    
    return discounts_pl, products_pl, employees_pl, stores_pl, customers_pl, transactions_pl, stats_summary

load_data_and_analyze()

📊 Loading data efficiently...
Loading discounts...
Loading products...
Loading employees...
Loading stores...
Loading customers...
Loading transactions (this may take a moment)...
Converting to Polars for better performance...
✅ Data loaded successfully!
Transactions shape: (6416827, 19)

📊 PERFORMING DESCRIPTIVE STATISTICS ANALYSIS

🏷️ DISCOUNTS Dataset Analysis:
----------------------------------------
   📋 Records: 181
   📊 Columns: 6
   💾 Memory: 0.02 MB

📦 PRODUCTS Dataset Analysis:
----------------------------------------
   📋 Records: 17,940
   📊 Columns: 12
   💾 Memory: 4.67 MB
   🏷️ Categories: 3

👥 EMPLOYEES Dataset Analysis:
----------------------------------------
   📋 Records: 404
   📊 Columns: 4
   💾 Memory: 0.02 MB

🏪 STORES Dataset Analysis:
----------------------------------------
   📋 Records: 35
   📊 Columns: 8
   💾 Memory: 0.00 MB

👤 CUSTOMERS Dataset Analysis:
----------------------------------------
   📋 Records: 1,643,306
   📊 Columns: 9
   💾 Memory: 169.71 MB

💳

C:\Users\Alan\AppData\Local\Temp\ipykernel_47356\164116053.py:32: DeprecationWarning: `NUMERIC_DTYPES` was deprecated in version 1.0.0. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  numeric_cols = discounts_pl.select(pl.col(pl.NUMERIC_DTYPES)).columns


   📋 Records: 6,416,827
   📊 Columns: 19
   💾 Memory: 936.64 MB

📈 OVERALL DATA SUMMARY:
----------------------------------------
   📊 Total Datasets: 6
   📋 Total Records: 8,078,693
   💾 Total Memory: 1111.06 MB

💾 Saving descriptive statistics...
✅ Descriptive statistics completed!
📁 Results saved in: descriptive_statistics/


(shape: (181, 6)
 ┌────────────┬────────────┬─────────┬─────────────────────┬───────────┬────────────────────────────┐
 │ Start      ┆ End        ┆ Discont ┆ Description         ┆ Category  ┆ Sub Category               │
 │ ---        ┆ ---        ┆ ---     ┆ ---                 ┆ ---       ┆ ---                        │
 │ str        ┆ str        ┆ f64     ┆ str                 ┆ str       ┆ str                        │
 ╞════════════╪════════════╪═════════╪═════════════════════╪═══════════╪════════════════════════════╡
 │ 2020-01-01 ┆ 2020-01-10 ┆ 0.4     ┆ 40% discount during ┆ Feminine  ┆ Coats and Blazers          │
 │            ┆            ┆         ┆ our New Ye…         ┆           ┆                            │
 │ 2020-01-01 ┆ 2020-01-10 ┆ 0.4     ┆ 40% discount during ┆ Feminine  ┆ Sweaters and Knitwear      │
 │            ┆            ┆         ┆ our New Ye…         ┆           ┆                            │
 │ 2020-01-01 ┆ 2020-01-10 ┆ 0.4     ┆ 40% discount during ┆ Masc

In [8]:
master_data, discounts_lookup = create_master_dataset()
# final_data = show_dataset_summary(master_data)

🏗️ Creating master dataset...
📊 Loading data efficiently...
Loading discounts...
Loading products...
Loading employees...
Loading stores...
Loading customers...
Loading transactions (this may take a moment)...
Converting to Polars for better performance...
✅ Data loaded successfully!
Transactions shape: (6416827, 19)
🔧 Preparing optimized lookup tables...
🗓️ Converting date columns to YYYY-MM-DD format: ['Start', 'End']
✅ Converted Start to YYYY-MM-DD date format
✅ Converted End to YYYY-MM-DD date format
🗓️ Converting date columns to YYYY-MM-DD format: ['Date Of Birth']
✅ Converted Date Of Birth to YYYY-MM-DD date format
✅ Lookup tables prepared
💰 Processing transactions...
🗓️ Converting date columns to YYYY-MM-DD format: ['Date']
✅ Converted Date to YYYY-MM-DD date format
💱 Adding USD conversion...
✅ Created Unit_Price_USD
✅ Created Line_Total_USD
✅ Created Invoice_Total_USD
🔗 Merging tables...
✅ Merged with products. Shape: (6416827, 29)
✅ Merged with customers. Shape: (6416827, 34)


In [9]:
master_data.shape

(6416827, 41)

In [12]:
m2 =master_data.unique()

In [13]:
m2.shape

(6416029, 41)

In [5]:
# Intial Transactions loaded: 6416827 rows
import pandas as pd
print("Loading customers...")
customers = pd.read_csv(r'C:\Users\Alan\Downloads\BDM\BDM_raw_data\transactions.csv', low_memory=False)
    

Loading customers...


In [6]:
customers['Customer ID'].nunique()

1283707